In [1]:
# !pip3 install beautifulsoup4
import pandas as pd
from bs4 import BeautifulSoup

In [15]:
df_20_test_features = pd.read_csv("../data/filtered20_responses_2023.csv")
df_unique_pages = df_20_test_features[['page', 'book']].drop_duplicates().reset_index(drop=True)
abc_book_name = 'College / Statistics and Data Science (ABC)'
abcd_book_name = 'College / Advanced Statistics and Data Science (ABCD)'
df_unique_pages_abc = df_unique_pages[df_unique_pages.book == abc_book_name].reset_index(drop=True)
df_unique_pages_abcd = df_unique_pages[df_unique_pages.book == abcd_book_name].reset_index(drop=True)

/var/folders/k8/d10c5tmd1b5g9_7g8dbmpt600000gp/T/ipykernel_92494/3163309484.py:1: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_20_test_features = pd.read_csv("../data/filtered20_responses_2023.csv")


# Parse HTML for table of contents for each book
- Identify each div on webpage of book table of content that contains chapter review (i.e. quiz)

### Currently Unused Features from Exercises. May Add Later
- Loop through unique pages for that book and extract the URL to that page, and add it to df
- Identify each div on webpage of book table of content that contains a page title or chapter review



In [71]:
def get_chapter_quiz(path):
    chapter_quiz_urls = {}
    with open(path, 'r') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the div with class "course-content"
    course_content_div = soup.find('div', class_='course-content')

    # If the course_content_div is found, find all anchor tags within it
    if course_content_div:
        # Find all 'a' tags with an href starting with the specified pattern
        import re
        for link in course_content_div.find_all('a', href=True):
            url = link['href']
            page_name = link.get_text(strip=True)
            if url.startswith("https://coursekata.org/preview/book/"):
                # Check if the page name contains "Chapter X Review" pattern
                if re.search(r"Chapter \d+ Review", page_name):
                    chapter_num = re.search(r"Chapter (\d+)", page_name).group(1)
                    if chapter_num not in chapter_quiz_urls:
                        chapter_quiz_urls[chapter_num] = [url]
                    else:
                        chapter_quiz_urls[chapter_num].append(url)
    return chapter_quiz_urls

# currently unused
def parse_page_urls(path):
    # import html file 
    with open(path, 'r') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the div with class "course-content"
    course_content_div = soup.find('div', class_='course-content')

    # Initialize an empty dictionary to store the results
    page_urls = {}

    # If the course_content_div is found, find all anchor tags within it
    if course_content_div:
        # Find all 'a' tags with an href starting with the specified pattern
        for link in course_content_div.find_all('a', href=True):
            url = link['href']
            if url.startswith("https://coursekata.org/preview/book/"):
                page_name = link.get_text(strip=True)
                page_urls[page_name] = url
    return page_urls

# helper
def add_urls_to_df(df, path):
    page_urls = parse_page_urls(path)
    df['url'] = None
    for i, row in df.iterrows():
        if row['page'] in page_urls:
            df.loc[i, 'url'] = page_urls[row['page']]
    return df

In [72]:
path_abcd = "table_of_contents/ABCD.html"
path_abc = "table_of_contents/ABC.html"

In [73]:
abc_chapter_quiz_urls = get_chapter_quiz(path_abc)
abcd_chapter_quiz_urls = get_chapter_quiz(path_abcd)

In [61]:
# currently unused
# df_unique_pages_abcd = add_urls_to_df(df_unique_pages_abcd, path_abcd)
# df_unique_pages_abc = add_urls_to_df(df_unique_pages_abc, path_abc)

# df_unique_pages_abc.to_csv("data/df_pages_url_abc.csv")
# df_unique_pages_abcd.to_csv("data/df_pages_url_abcd.csv")

# Get Text + Images from Chapter Quiz

In [79]:
from bs4 import BeautifulSoup

def extract_question_content(html):
    soup = BeautifulSoup(html, "html.parser")
    # Extract all visible text from the question area
    # We'll get all text in .learnosity-item-page (excluding script/style/hidden/iframe/etc)
    main_div = soup.find("div", class_="learnosity-item-page")
    if not main_div:
        # Fallback: use the whole document
        main_div = soup

    # Remove all script, style, iframe, and visually-hidden elements
    for tag in main_div(['script', 'style', 'iframe', 'noscript', 'input']):
        tag.decompose()
    # Optionally remove 'sr-only' and aria-hidden text
    for tag in main_div.find_all(class_=["sr-only"]):
        tag.decompose()
    for tag in main_div.find_all(attrs={"aria-hidden": "true"}):
        tag.decompose()

    # Get visible text
    text = main_div.get_text(separator=' ', strip=True)

    # Find all image URLs in the question
    image_urls = [img['src'] for img in main_div.find_all('img') if img.get('src')]

    return {"text": text, "image_urls": str(image_urls)}

def agg_question_content_per_url(iframe_contents):
    text = ""
    image_urls = []

    for iframe_id, html_body in iframe_contents.items():
        result = extract_question_content(html_body)
        text += f"\nQuestion ID: {iframe_id}\n{result['text']}"
        image_urls.extend(eval(result['image_urls']))

    # make image_urls unique
    image_urls = list(set(image_urls))

    return {"text": text, "image_urls": str(image_urls)}

def combine_multiple_question_content(question_content_list):
    if len(question_content_list) == 1:
        return question_content_list[0]
    text = ""
    image_urls = []
    for i in range(len(question_content_list)):
        text += f"\nChapter Quiz Page {i+1}:\n{question_content_list[i]['text']}"
        image_urls.extend(eval(question_content_list[i]['image_urls']))
    image_urls = list(set(image_urls))
    return {"text": text, "image_urls": str(image_urls)}


In [83]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

def get_chapter_quiz_page_content(chapter_urls: dict):
    page_content = {
        "chapter" : [],
        "urls" : [],
        'text': [],
        'images': [],
    }

    # 1. Launch browser
    driver = webdriver.Chrome()  # or set options if needed
    for chap, urls in chapter_urls.items():
        if int(chap) > 9:
            continue
        
        page_content['chapter'].append(chap)
        page_content['urls'].append(urls)

        # There may be multiple URLs for the same chapter
        question_content_list = []
        for i in range(len(urls)):
            url = urls[i]
            # 2. Load your page
            driver.get(url)  # Replace with your actual URL

            # 3. Wait for at least one iframe to appear (increase timeout if needed)
            time.sleep(5)  # Explicitly wait for 5 seconds
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "iframe"))
            )

            # 4. Gather all iframes
            iframes = driver.find_elements(By.TAG_NAME, "iframe")
            iframe_contents = {}
            for iframe in iframes:
                iframe_id = iframe.get_attribute('id')
                if iframe_id:
                    # 5. Switch to this iframe
                    driver.switch_to.frame(iframe)
                    # 6. Wait for something meaningful to load, e.g. body > *
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.TAG_NAME, "body"))
                    )
                    # 7. Extract only the body content from the iframe
                    body_element = driver.find_element(By.TAG_NAME, "body")
                    body_html = body_element.get_attribute('innerHTML')
                    
                    # Store the full iframe HTML in the dictionary for reference if needed
                    iframe_contents[iframe_id] = body_html
                    
                    # 8. Switch back to the main page before handling the next iframe
                    driver.switch_to.default_content()
            print(f"Combined HTML from {len(iframes)} iframes in chapter {chap}")

            # Parse out text and image content per URL
            question_content_per_url = agg_question_content_per_url(iframe_contents)
            question_content_list.append(question_content_per_url)

        # Consolidate text and image content per chapter
        question_content = combine_multiple_question_content(question_content_list)
        page_content['text'].append(question_content['text'])
        page_content['images'].append(question_content['image_urls'])
        page_content['images'] = list(set(page_content['images']))
    driver.quit()

    return page_content

In [84]:
abc_chapter_quiz_page_content = get_chapter_quiz_page_content(abc_chapter_quiz_urls)
abcd_chapter_quiz_page_content = get_chapter_quiz_page_content(abcd_chapter_quiz_urls)

Combined HTML from 16 iframes in chapter 1
Combined HTML from 22 iframes in chapter 2
Combined HTML from 18 iframes in chapter 2
Combined HTML from 24 iframes in chapter 3
Combined HTML from 16 iframes in chapter 3
Combined HTML from 23 iframes in chapter 4
Combined HTML from 17 iframes in chapter 4
Combined HTML from 23 iframes in chapter 5
Combined HTML from 17 iframes in chapter 5
Combined HTML from 23 iframes in chapter 6
Combined HTML from 14 iframes in chapter 6
Combined HTML from 15 iframes in chapter 7
Combined HTML from 23 iframes in chapter 8
Combined HTML from 23 iframes in chapter 9
Combined HTML from 14 iframes in chapter 9
Combined HTML from 21 iframes in chapter 10
Combined HTML from 22 iframes in chapter 10
Combined HTML from 18 iframes in chapter 11
Combined HTML from 23 iframes in chapter 11
Combined HTML from 24 iframes in chapter 12
Combined HTML from 23 iframes in chapter 12
Combined HTML from 16 iframes in chapter 1
Combined HTML from 22 iframes in chapter 2
Combi

In [100]:
abc_book_name = 'College / Statistics and Data Science (ABC)'
abcd_book_name = 'College / Advanced Statistics and Data Science (ABCD)'

df_abc_chapter_quiz_page_content = pd.DataFrame(abc_chapter_quiz_page_content)
df_abcd_chapter_quiz_page_content = pd.DataFrame(abcd_chapter_quiz_page_content)

df_abc_chapter_quiz_page_content['book'] = abc_book_name
df_abcd_chapter_quiz_page_content['book'] = abcd_book_name

df_chapter_quiz_page_content_all = pd.concat([df_abc_chapter_quiz_page_content, df_abcd_chapter_quiz_page_content]).reset_index(drop=True)
df_chapter_quiz_page_content_all.to_csv("data/chapter_quiz_page_content_all.csv")

# Get Text + Images from Exercises (currently unused)

In [14]:
import requests

def get_page_text_and_images(url):
    # request page
    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html.parser')
    student_content = soup.select_one('div.course-content.student')

    # Find all img and alt text tags
    img_tags = soup.find_all('img')
    alt_text_and_images = []
    images_only = []
    for img in img_tags:
        alt_text_and_images.append((img.get('alt', ''), img.get('src', '')))
        images_only.append(img.get('src', ''))

    page_content = {
        'text': student_content.text.strip() if student_content else None,
        'images_only': str(images_only),
        'alt_text_and_images': str(alt_text_and_images)
    }

    return page_content

def get_page_content_list_chapter_urls(chapter_urls: dict):
    page_content = {
        "chapter" : [],
        "urls" : [],
        'text': [],
        'images_only': [],
        'alt_text_and_images': []
    }
    for chap, urls in chapter_urls.items():
        page_content['chapter'].append(chap)
        page_content['urls'].append(urls)
        for i in range(len(urls)):
            url = urls[i]
            url_page_content = get_page_text_and_images(url)
            page_num = i + 1 
            page_content['text'] = f"\nChapter Quiz Page {page_num}:\n" + url_page_content['text']
            if page_num == 1:
                page_content['images_only'] = url_page_content['images_only']
                page_content['alt_text_and_images'] = url_page_content['alt_text_and_images']
            else:
                page_content['images_only'] = str(eval(page_content['images_only']) + eval(url_page_content['images_only']))
                page_content['alt_text_and_images'] = str(eval(page_content['alt_text_and_images']) + eval(url_page_content['alt_text_and_images']))

    return page_content


def add_page_content_to_df(df):
    df['page_context_text'] = None
    df['page_context_alt_text_and_images'] = None
    for i, row in df.iterrows():
        if  row['url']:
            page_content = get_page_text_and_images(row['url'])
            df.loc[i, 'page_context_text'] = page_content['text']
            df.loc[i, 'page_context_images_only'] = page_content['images_only']
            df.loc[i, 'page_context_alt_text_and_images'] = page_content['alt_text_and_images']
    return df



In [21]:
abc_chapter_quiz_page_content = get_page_content_list_chapter_urls(abc_chapter_quiz_urls)
abcd_chapter_quiz_page_content = get_page_content_list_chapter_urls(abc_chapter_quiz_urls)

df_abc_chapter_quiz_page_content = pd.DataFrame(abc_chapter_quiz_page_content)
df_abcd_chapter_quiz_page_content = pd.DataFrame(abcd_chapter_quiz_page_content)

df_abc_chapter_quiz_page_content['book'] = abc_book_name
df_abcd_chapter_quiz_page_content['book'] = abcd_book_name

df_chapter_quiz_page_content_all = pd.concat([df_abc_chapter_quiz_page_content, df_abcd_chapter_quiz_page_content]).reset_index(drop=True)
df_chapter_quiz_page_content_all.to_csv("data/chapter_quiz_page_content_all.csv")

In [74]:
df_unique_pages_abc = add_page_content_to_df(df_unique_pages_abc)
df_unique_pages_abcd = add_page_content_to_df(df_unique_pages_abcd)

df_unique_pages_abc.to_csv("data/df_pages_content_abc.csv")
df_unique_pages_abcd.to_csv("data/df_pages_content_abcd.csv")